In [1]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\flipi\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
import pandas as pd
import string
from datetime import datetime
import re
from collections import OrderedDict

In [3]:
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import PorterStemmer
from spellchecker import SpellChecker
from nltk.corpus import words

In [4]:
nickname = 'gustavo_abroad'
channel = 'greekgodx' 

In [5]:
twitchEmotesToUse = ["OMEGALUL", "LULW" , "monkaW", "WAYTOODANK","EZ","Clap","LUL","BOOBA", 
                     "Pepega", "FeelsGoodMan", "WutFace", "Sadge", "Kreygasm", "Kappa", "peepoSad", "PogU", "TRUEING", "monkaS"]

In [6]:
#Function to remove punctuation
#def message_cleaning(message):
    #Test_punc_removed = [char for char in message if char not in string.punctuation]
    #Test_punc_removed_join = ''.join(Test_punc_removed)
    #return Test_punc_removed_join

In [7]:
def message_cleaning_duplicate(message):
    messageToshow = ''
    messagelist = list(OrderedDict.fromkeys(message.split(' ')))
    for word in messagelist:
         messageToshow = messageToshow + word + " "
    return messageToshow

In [8]:
def add_element_to_list(time_logged_to_add,channel_to_add,username_message_to_add,messageToshow_to_add,list_to_add):
    tag_owner = 0
    if(('@' + channel) in messageToshow_to_add):
                                    tag_owner = 1
    d = {
            'Date': time_logged_to_add,
            'Channel': channel_to_add,
            'Username': username_message_to_add,
            'Message': messageToshow_to_add,
            'Tag_owner':tag_owner
        }
    list_to_add.append(d)

In [9]:
def get_chat_dataframe(file):
    data = []

    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n\n\n')
        for line in lines:
            try:
                time_logged = line.split('—')[0].strip()
                if(time_logged != ''):
                    time_logged = datetime.strptime(time_logged, '%Y-%m-%d_%H:%M:%S')

                username_message = line.split('—')[1:]
                infoInTheSameTime = username_message[0].split('\n\n')[1:]
                if(infoInTheSameTime):
                    firstMultiInfo = 0
                    for otherChat in infoInTheSameTime:
                        if(not firstMultiInfo):
                            messageToshow = username_message[0].split('\n\n')[0].split('#')[1:]
                            if(messageToshow):
                                channel = messageToshow[0].split(':')[0]
                                messageToshow = messageToshow[0].split(str(channel) + '' + ':')[1:][0]
                                username_message = username_message[0].split(':')[1:]
                                username_message = username_message[0].split('!')[0]
                                add_element_to_list(time_logged,channel,username_message,messageToshow,data)
                                firstMultiInfo = 1
                        messageToshow = ''
                        otherChatAtSameTime = otherChat.split(':')[1:]
                        username_message = otherChatAtSameTime[0].split('!')[0]
                        channel = otherChatAtSameTime[0].split('!')[1:]
                        for othermessage in otherChatAtSameTime[1:]:
                            messageToshow = messageToshow + '' + othermessage
                        if(channel and messageToshow and (nickname not in username_message)):
                            channel = channel[0].split('#')[1:][0]
                            add_element_to_list(time_logged,channel,username_message,messageToshow,data)

                else:
                    messageToshow = username_message[0].split('#')[1:]
                    channel = messageToshow[0].split(':')[0]
                    messageToshow = messageToshow[0].split(str(channel) + '' + ':')[1:][0]
                    username_message = username_message[0].split(':')[1:]
                    username_message = username_message[0].split('!')[0]
                    add_element_to_list(time_logged,channel,username_message,messageToshow,data)
            
            except Exception:
                pass
            
    return pd.DataFrame().from_records(data)

In [10]:
df = get_chat_dataframe('ChatLogs/chatgreekgodx091421.log')

In [11]:
print(df.shape)

(14065, 5)


In [12]:
df.head(15)

,Date,Channel,Username,Message,Tag_owner
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0


In [13]:
df.to_csv('ChatLogs/chatgreekgodx091421.csv', index=False) 

Now, we will remove the message that only have one character, because this kind of messages are not relevant

In [14]:
#Function to remove punctuation
#messageClean = df['Message'].apply(message_cleaning)
#df['clean_message'] = messageClean
#print(df.shape)
#df.head(15)   

In [15]:
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [16]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["Cleanest_message"] = df["Message"].apply(lambda text: remove_stopwords(text))
df.head(15)

,Date,Channel,Username,Message,Tag_owner,Cleanest_message
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,SMOrc SMOrc SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,Alarm turn volume Alarm
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,SMOrc SMOrc


In [17]:
message_clean_duplicates = df['Cleanest_message'].apply(message_cleaning_duplicate)
df['clean_message_duplicates'] = message_clean_duplicates

In [18]:
df.head(15)

,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,Pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,Irl stream snipers,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kicked Pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepeMeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,SMOrc SMOrc SMOrc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,FeelsDankMan,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,SMOrc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,GOBLIN,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,Alarm turn volume Alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,SMOrc SMOrc,SMOrc


In [19]:
message_to_remove_list = []
for message in df['Message']:
    words_not_duplicate = ' '.join(set(message))
    if(len(words_not_duplicate) <= 1):
        message_to_remove = df[df['Message'] == message]
        for same_messages in message_to_remove.values:
            d = {
                'Data': same_messages[0],
                'Channel': same_messages[1],
                'Username': same_messages[2],
                'Message': same_messages[3]
            }
            if d not in message_to_remove_list:
                message_to_remove_list.append(d)
            if ((df['Message'] == message) & (df['Username'] == same_messages[2])).any():
                df.drop(df[(df['Message'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)
        
pd_message_to_remove =pd.DataFrame().from_records(message_to_remove_list) 

print(pd_message_to_remove.shape)
pd_message_to_remove.head(15)


(305, 4)


,Data,Channel,Username,Message
0,2021-09-14 17:51:48,greekgodx,kissalanpojat,","
1,2021-09-14 17:52:17,greekgodx,stanmeister,D
2,2021-09-14 17:52:30,greekgodx,palenkoe_man,D
3,2021-09-14 17:52:34,greekgodx,gabenator,D
4,2021-09-14 17:54:47,greekgodx,kaecilius13,D
5,2021-09-14 17:54:54,greekgodx,richkkonasavage93,D
6,2021-09-14 17:55:03,greekgodx,midouuuuuuuuuuuuuuuuuuuu,D
7,2021-09-14 17:55:22,greekgodx,fletchdawg,D
8,2021-09-14 17:55:26,greekgodx,spike_o5,D
9,2021-09-14 17:55:28,greekgodx,kissalanpojat,D


We see that we remove 305 columns

In [20]:
print(df.shape)
df.head(15)

(13760, 7)


,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,Pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,Irl stream snipers,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kicked Pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepeMeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,SMOrc SMOrc SMOrc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,FeelsDankMan,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,SMOrc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,GOBLIN,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,Alarm turn volume Alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,SMOrc SMOrc,SMOrc


Now, we need to remove spam links for bots/ persons

In [21]:
cnt = Counter()
for text in df["clean_message_duplicates"].values:
    for word in text.split():
        cnt[word] += 1
        
mostCommon = pd.DataFrame.from_dict(cnt, orient='index').reset_index()
mostCommon.head(15)

,index,0
0,Pepega,223
1,Irl,5
2,stream,30
3,snipers,3
4,v,4
5,tec,1
6,kicked,2
7,Pog,49
8,pepeMeltdown,30
9,SMOrc,65


In [22]:
mostCommon.to_csv('ChatLogs/mostCommonWords.csv', index=False) 

In [23]:
n_rare_words = 20
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
RAREWORDS

{'AYEEE',
 'DAM',
 'DMX',
 'Lordy',
 'MOnkaW',
 'OMEGALULU',
 'PAUSEEE',
 'Summit',
 'TYPED',
 'WOLF',
 'YUUUUUUUUUUH',
 'badly',
 'deep?',
 'freeze',
 'lord',
 'lordt',
 'losDababy',
 'numb',
 'powerwalk',
 'roof,'}

In [24]:
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

In [26]:
df["Cleanest_message"] = df["Cleanest_message"].apply(lambda text: remove_rarewords(text))
df["clean_message_duplicates"] = df["clean_message_duplicates"].apply(lambda text: remove_rarewords(text))
df.head(15)

,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,Pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,Irl stream snipers,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kicked Pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepeMeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,SMOrc SMOrc SMOrc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,FeelsDankMan,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,SMOrc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,GOBLIN,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,Alarm turn volume Alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,SMOrc SMOrc,SMOrc


In [27]:
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

In [28]:
df["Cleanest_message"] = df["Cleanest_message"].apply(lambda text: stem_words(text))
df.head(15)

,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepemeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,smorc smorc smorc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,feelsdankman,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,smorc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,goblin,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,smorc smorc,SMOrc


In [29]:
spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    print(misspelled_words)
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [30]:
#df["Cleanest_message"] = df["Cleanest_message"].apply(lambda text: correct_spellings(text))
df.head(15)

,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepemeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,smorc smorc smorc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,feelsdankman,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,smorc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,goblin,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,smorc smorc,SMOrc


In [31]:
message_to_remove_list = []
for message in df['Message']:
    if(('https' in message) or  ('.com' in message)):
        message_to_remove = df[df['Message'] == message]
        for same_messages in message_to_remove.values:
            d = {
                'Channel': same_messages[1],
                'Username': same_messages[2],
                'Message': same_messages[3]
            }
            message_to_remove_list.append(d)
            if ((df['Message'] == message) & (df['Username'] == same_messages[2])).any():
                df.drop(df[(df['Message'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)

pd_message_to_remove =pd.DataFrame().from_records(message_to_remove_list) 

print(pd_message_to_remove.shape)
pd_message_to_remove.head(15)

(20, 3)


,Channel,Username,Message
0,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
1,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
2,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
3,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
4,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
5,greekgodx,streamlabs,Check out the Official GreekGodx subreddit red...
6,greekgodx,streamlabs,Follow Greek on Twitter greekW Main: twitter.c...
7,greekgodx,streamlabs,Follow Greek on Twitter greekW Main: twitter.c...
8,greekgodx,streamlabs,Follow Greek on Twitter greekW Main: twitter.c...
9,greekgodx,streamlabs,Follow Greek on Twitter greekW Main: twitter.c...


In [32]:
print(df.shape)
df.head(15)

(13740, 7)


,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepemeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,smorc smorc smorc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,feelsdankman,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,smorc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,goblin,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,smorc smorc,SMOrc


Now, we need to remove rows that contain only numbers, because they will be useless

In [33]:
message_to_remove_list = []
for message in df['Message']:
    if(message.isnumeric() or ((message.replace('-', '')).replace('.', '')).isnumeric()):
        message_to_remove = df[df['Message'] == message]
        for same_messages in message_to_remove.values:
            d = {
                'Channel': same_messages[1],
                'Username': same_messages[2],
                'Message': same_messages[3]
            }
            message_to_remove_list.append(d)
            if ((df['Message'] == message) & (df['Username'] == same_messages[2])).any():
                df.drop(df[(df['Message'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)

pd_message_to_remove =pd.DataFrame().from_records(message_to_remove_list) 

print(pd_message_to_remove.shape)
pd_message_to_remove.head(15)

(15, 3)


,Channel,Username,Message
0,greekgodx,themuleus,-1
1,greekgodx,hannes___________,-1
2,greekgodx,richkkonasavage93,-1
3,greekgodx,okako01,-1
4,greekgodx,invert3brat,-1
5,greekgodx,mindshape,-1
6,greekgodx,vitacrown,-1
7,greekgodx,flamehotsoda,-1
8,greekgodx,scropionxking,-1
9,greekgodx,bnicerguy,-1


In [35]:
print(df.shape)
df.head(15)

(13725, 7)


,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepemeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,smorc smorc smorc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,feelsdankman,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,smorc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,goblin,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,smorc smorc,SMOrc


In [36]:
message_to_remove_list = []
for message in df['clean_message_duplicates']:
    if(len(message.split()) <= 1):
        if(len(message.split()) == 0):
            message_to_remove = df[df['clean_message_duplicates'] == message]
            for same_messages in message_to_remove.values:
                d = {
                    'Data': same_messages[0],
                    'Channel': same_messages[1],
                    'Username': same_messages[2],
                    'Message': same_messages[3]
                }
                if d not in message_to_remove_list:
                    message_to_remove_list.append(d)
                if ((df['clean_message_duplicates'] == message) & (df['Username'] == same_messages[2])).any():
                    df.drop(df[(df['clean_message_duplicates'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)

        
pd_message_to_remove =pd.DataFrame().from_records(message_to_remove_list) 

print(pd_message_to_remove.shape)
pd_message_to_remove.head(15)


(54, 4)


,Data,Channel,Username,Message
0,2021-09-14 17:52:00,greekgodx,suddym8,it was me
1,2021-09-14 17:53:06,greekgodx,papiatnehotpafi,its over
2,2021-09-14 17:53:14,greekgodx,fluxenis,yours
3,2021-09-14 17:53:33,greekgodx,leeqox,no
4,2021-09-14 17:53:37,greekgodx,adamev1,no
5,2021-09-14 17:55:02,greekgodx,select87,what you doing you
6,2021-09-14 17:55:42,greekgodx,invert3brat,me
7,2021-09-14 17:58:47,greekgodx,dz69420,do it
8,2021-09-14 18:06:47,greekgodx,aalert,it's over
9,2021-09-14 18:13:42,greekgodx,lew_____,no


In [37]:
print(df.shape)
df.head(15)

(13671, 7)


,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
3,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown,0,pepemeltdown,pepeMeltdown
4,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc,0,smorc smorc smorc,SMOrc
5,2021-09-14 17:51:10,greekgodx,5zipp,FeelsDankMan,0,feelsdankman,FeelsDankMan
6,2021-09-14 17:51:10,greekgodx,mrdude90,SMOrc,0,smorc,SMOrc
7,2021-09-14 17:51:10,greekgodx,bilaal44,GOBLIN,0,goblin,GOBLIN
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc,0,smorc smorc,SMOrc


In [42]:
message_to_remove_list = []
for message in df['clean_message_duplicates']:
    if(len(message.split()) <= 1):
        if(not (message.split()[0] in words.words()) and (not(message.split()[0] in twitchEmotesToUse))):
            message_to_remove = df[df['clean_message_duplicates'] == message]
            for same_messages in message_to_remove.values:
                d = {
                    'Data': same_messages[0],
                    'Channel': same_messages[1],
                    'Username': same_messages[2],
                    'Message': same_messages[3]
                }
                if d not in message_to_remove_list:
                    message_to_remove_list.append(d)
                if ((df['clean_message_duplicates'] == message) & (df['Username'] == same_messages[2])).any():
                    df.drop(df[(df['clean_message_duplicates'] == message) & (df['Username'] == same_messages[2])].index, inplace=True)

        
pd_message_to_remove =pd.DataFrame().from_records(message_to_remove_list) 

print(pd_message_to_remove.shape)
pd_message_to_remove.head(15)

(3895, 4)


,Data,Channel,Username,Message
0,2021-09-14 17:51:10,greekgodx,no_much,pepeMeltdown
1,2021-09-14 17:53:37,greekgodx,dreammistm,pepeMeltdown
2,2021-09-14 18:06:29,greekgodx,yoourself,pepeMeltdown
3,2021-09-14 18:06:36,greekgodx,boozool,pepeMeltdown
4,2021-09-14 18:06:37,greekgodx,ghos7dog,pepeMeltdown
5,2021-09-14 18:06:47,greekgodx,roeltjaa,pepeMeltdown
6,2021-09-14 18:12:46,greekgodx,zombie_guy94,pepeMeltdown
7,2021-09-14 18:14:20,greekgodx,mostafa__mg,pepeMeltdown
8,2021-09-14 18:22:00,greekgodx,ggx_goblin,pepeMeltdown
9,2021-09-14 17:51:10,greekgodx,alundz0114,SMOrc SMOrc SMOrc


In [43]:
print(df.shape)
df.head(15)

(9776, 7)


,Date,Channel,Username,Message,Tag_owner,Cleanest_message,clean_message_duplicates
0,2021-09-14 17:51:09,greekgodx,mustfax19,Pepega,0,pepega,Pepega
1,2021-09-14 17:51:09,greekgodx,bangkeys,Irl stream snipers,0,irl stream sniper,Irl stream snipers
2,2021-09-14 17:51:09,greekgodx,exiledxinsanity,v tec kicked in Pog,0,v tec kick pog,v tec kicked Pog
8,2021-09-14 17:51:10,greekgodx,leeqox,Alarm turn up the volume Alarm,0,alarm turn volum alarm,Alarm turn volume
11,2021-09-14 17:51:10,greekgodx,adamev1,Pepega Clap,0,pepega clap,Pepega Clap
14,2021-09-14 17:51:11,greekgodx,proxe0,he is high,0,high,high
17,2021-09-14 17:51:12,greekgodx,spr1nt3r,THE INSANE SAILOR monkaS,0,the insan sailor monka,THE INSANE SAILOR monkaS
18,2021-09-14 17:51:12,greekgodx,ahmet_charts,"dont steer hard, STEER JUST A LITTLE",0,"dont steer hard, steer just a littl","dont steer hard, STEER JUST A LITTLE"
20,2021-09-14 17:51:12,greekgodx,badmindsetabuser,go left bitch,0,go left bitch,go left bitch
22,2021-09-14 17:51:13,greekgodx,fartecho,He ius turning,0,he iu turn,He ius turning


In [44]:
df.to_csv('ChatLogs/chatgreekgodx091421Clean.csv', index=False) 

In [45]:
pd_message_to_remove.to_csv('ChatLogs/test.csv', index=False) 